# environment setup

### Requirements

`Python>=3.9` with the following:

```
napari[pyQT5]
napari-simpleitk-image-processing
jupyterlab
```

and normally:

```
yt[full]
yt-napari 
```

(but not right now...)

e.g., using pyenv-virtualenv

```
$ pyenv virtualenv 3.10.11 ytnapari_310
$ pyenv activate ytnapari_310
$ pip install -r requirements.txt
```





# basic usage 

## standalone gui, gui overview 

from command line

```
$ napari
```

GUI overview: 
* top left, individual settings for a layer
* bot left, list of available layers
* right: the viewer

Load a sample:

    File -> Open Sample -> -> cells 3d (2 Channel) 

Point out the two layers 

2d view: bottom slider through 3rd dimension

    adjust colormaps, limits of individual layers
    final image on right controlled by blending of layers: top-down ordering of image layers 

Switch to 3d: 

    additional rendering optoins: mip, etc.
    depiction: plane, shift click to drag

# A quick image segmentation and sampling exercise


In [1]:
import napari
v = napari.Viewer()

Assistant skips harvesting pyclesperanto as it's not installed.
Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


Load a sample:

    File -> Open Sample -> -> cells 3d (2 Channel) 


Create a labeled image: void detection on the membrane layer 

    tools -> filtering -> rescale 0/1
    tools -> segmentation/labeling -> morphological watersheld on rescaled membrane, 0.01 isn't bad... 
    click on labels layer, check show selected, click through in 3d to see the different labels 
    tools -> measurement tables -> object feature properties (scikit-image)
    click intensity, select nuclei 
    

In [2]:
v.layers

[<Image layer 'membrane' at 0x7f9fa92950d0>, <Image layer 'nuclei' at 0x7f9fa8e2ee80>, <Image layer 'Result of rescale_intensity' at 0x7f9fa575fe80>, <Labels layer 'Result of morphological_watershed' at 0x7f9fa8bcdc40>]

In [3]:
result = v.layers[-1]

In [4]:
result.properties.keys()

dict_keys(['label', 'max_intensity', 'mean_intensity', 'min_intensity', 'standard_deviation_intensity', 'index'])

In [5]:
import pandas as pd 

df = pd.DataFrame(result.properties)
df

,label,max_intensity,mean_intensity,min_intensity,standard_deviation_intensity,index
0,1,35686.0,2685.205339,808.0,1461.423026,1
1,2,21480.0,2869.887147,868.0,1393.035166,2
2,3,9549.0,2175.639100,461.0,1130.921289,3
3,4,28211.0,4297.006995,2180.0,2058.409979,4
4,5,26461.0,4197.802721,2095.0,1755.581076,5
...,...,...,...,...,...,...
158,159,10349.0,5253.756098,3167.0,1576.353535,159
159,160,13503.0,5715.134969,2198.0,2915.814867,160
160,161,23717.0,6230.320856,2781.0,2881.632131,161
161,162,42948.0,1798.602978,0.0,1247.637894,162


# anatomy of a plugin: yt-napari 

yt-napari: 

load data into napari from yt
yt-napari: creates samples of yt datasets, loads in as image arrays

* json loader
* GUI loader
* additional notebook helper functions

In [6]:
v.layers.clear()

yt : [INFO     ] 2024-02-09 10:18:18,746 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2024-02-09 10:18:18,746 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2024-02-09 10:18:18,747 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-02-09 10:18:18,748 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-02-09 10:18:18,748 Parameters: cosmological_simulation   = 0

Parsing Hierarchy : 100%|███████████████████████████████████████████████████████████████| 173/173 [00:00<00:00, 1370.57it/s]
yt : [INFO     ] 2024-02-09 10:18:18,886 Gathering a field list (this may take a moment.)


look at the json

working with simulation data, **not** strict image data. 

So the JSON here specifies, not just a file, but information on how to sample from that file. 